In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

### <font color=blue> Get ground truth inc deaths at state level and clean the data.

In [2]:
from epsampling.utils import drop_sers_with_nans

fpath = os.path.join(DATA_DIR,'raw', 'COVIDhub-truth' ,'truth_nytimes-Incident_Deaths_NEW.txt')
df = pd.read_csv(fpath)
df = drop_sers_with_nans(df, from_axis='rows')

## REFORMAT dataframe ...
## rename cols
df.rename({'value':'True_inc_state_deaths', 'location':'state_fips'},axis=1,inplace=True)
## capitalize cols.
df.columns = df.columns.str.capitalize()
## choose cols
df = df[['State_fips','Date','True_inc_state_deaths']]

## IMPORTANT: choose only state fips
df = df[df.State_fips!='US']

## convert all cols to numerical
df['State_fips'] = df.State_fips.astype(int)
df['Date'] = df.Date.apply(lambda x: date_str_to_int(x))

## Drop negative inc deaths lol.
df = df[df.True_inc_state_deaths>=0]
display(df)

Dropped rows with NaNs!
Num rows before: 2548034
Num rows after: 2466751


,State_fips,Date,True_inc_state_deaths
0,1,20200313,0.0
1,1,20200314,0.0
2,1,20200315,0.0
3,1,20200316,0.0
4,1,20200317,0.0
...,...,...,...
2465902,78030,20220509,0.0
2465903,78030,20220510,0.0
2465904,78030,20220511,0.0
2465905,78030,20220512,0.0


In [3]:
## IMPORTANT: pull out samples from 'nytimes' that have matched dates to 'COVIDhub-ensemble' ...
timestamp = '20241001-223952'
fpath = os.path.join(DATA_DIR,'processed', f'formatted_COVIDhub-ensemble_{timestamp}.csv')
df_hub = pd.read_csv(fpath)

df_hub,_ = load_csv('formatted_COVIDhub-ensemble')
display(df_hub)

my_dates = df_hub.Date.unique().tolist()
df = df[df.Date.isin(my_dates)]
df.reset_index(inplace=True,drop=True)
# display(df)

## Join proj df with truth df.
df = df.merge(df_hub,on=['State_fips','Date'])
display(df)

,State_fips,Date,Proj_inc_deaths
0,66,20200613,0.084503
1,69,20200613,0.410538
2,72,20200613,4.539067
3,78,20200613,0.123634
4,11,20200613,22.548359
...,...,...,...
8054,72,20230311,19.000000
8055,60,20230311,0.000000
8056,66,20230311,0.000000
8057,69,20230311,0.000000


,State_fips,Date,True_inc_state_deaths,Proj_inc_deaths
0,1,20200613,4.0,69.729674
1,1,20200620,16.0,83.302156
2,1,20200627,12.0,68.179357
3,1,20200704,1.0,76.238975
4,1,20200711,10.0,88.362530
...,...,...,...,...
5504,78,20220409,0.0,1.000000
5505,78,20220416,0.0,0.000000
5506,78,20220423,0.0,0.000000
5507,78,20220430,0.0,0.000000


In [4]:
## Now need to get state population ...

### LOAD CSV
timestamp = '20241002-004543'
fpath = os.path.join(DATA_DIR,'processed', f'processed_naive_deaths_{timestamp}.csv')
df_deaths = pd.read_csv(fpath, usecols=['State_fips','State_pop'])
df_deaths.drop_duplicates(inplace=True)
df_deaths.reset_index(inplace=True,drop=True)
df_deaths 

## Join proj df with truth df.
df = df.merge(df_deaths,on=['State_fips'])
display(df)

,State_fips,Date,True_inc_state_deaths,Proj_inc_deaths,State_pop
0,1,20200613,4.0,69.729674,4997675
1,1,20200620,16.0,83.302156,4997675
2,1,20200627,12.0,68.179357,4997675
3,1,20200704,1.0,76.238975,4997675
4,1,20200711,10.0,88.362530,4997675
...,...,...,...,...,...
4971,56,20220409,0.0,8.000000,576641
4972,56,20220416,0.0,5.000000,576641
4973,56,20220423,0.0,4.000000,576641
4974,56,20220430,0.0,4.000000,576641


In [5]:
## SAVE CSV!
# fpath = os.path.join(DATA_DIR,'processed','scratch',f'formatted_COVIDhub-ensemble_{DT}.csv')
fpath = os.path.join(DATA_DIR,'processed','scratch', f'formatted_state_inc_deaths_{DT}.csv')
df.to_csv(fpath,index=False)
print(DT)

20241004-192756


In [ ]:
df.True_inc_state_deaths.mean()

In [ ]:
df.True_inc_state_deaths.unique()

In [ ]:

## IMPORTANT: pull out samples from 'nytimes' that have matched dates to 'COVIDhub-ensemble' ...
timestamp = '20241001-223952'
fpath = os.path.join(DATA_DIR,'processed', f'formatted_COVIDhub-ensemble_{timestamp}.csv')
df_hub = pd.read_csv(fpath)

df_hub,_ = load_csv('formatted_COVIDhub-ensemble')
display(df_hub)

# my_dates = df_hub.Date.unique().tolist()
# df = df[df.Date.isin(my_dates)]
# df.reset_index(inplace=True,drop=True)
# # display(df)

# ## Join proj df with truth df.
# df = df.merge(df_hub,on=['State_fips','Date'])
# display(df)

In [ ]:
df_hub.Proj_inc_deaths.unique()

In [ ]:
## Now need to get state population ...

### LOAD CSV
timestamp = '20241002-004543'
fpath = os.path.join(DATA_DIR,'processed', f'processed_naive_deaths_{timestamp}.csv')
df_deaths = pd.read_csv(fpath, usecols=['State_fips','State_pop'])
df_deaths.drop_duplicates(inplace=True)
df_deaths.reset_index(inplace=True,drop=True)
df_deaths 

## Join proj df with truth df.
df = df.merge(df_deaths,on=['State_fips'])
display(df)

In [ ]:
## SAVE CSV!
# fpath = os.path.join(DATA_DIR,'processed','scratch',f'formatted_COVIDhub-ensemble_{DT}.csv')
fpath = os.path.join(DATA_DIR,'processed','scratch', f'formatted_state_inc_deaths_{DT}.csv')
df.to_csv(fpath,index=False)
print(DT)